In [1]:
import wandb
import pandas as pd
import numpy as np
import texttable as tt
import latextable

In [12]:
EXPERIMENT_GROUP = "exp-5-data-augmentation"
BASELINE_SETUP = "no-aug"
AVERAGE_EPOCHS = None # 5 # or None | must be odd
HAS_WEIGHTED_ACCURACY = True

In [13]:
def get_best_epoch_metrics(run, metric_name, average_epochs=None):
    history = run.history(samples=10000)
    
    # Find the epoch with the best metric value
    best_epoch = history[metric_name].idxmax()
    best_metrics = history.loc[best_epoch]
    
    if average_epochs:
        # Find indices of valid (non-NaN) values
        valid_indices = history[metric_name].dropna().index
        
        # Locate the best epoch index in the valid indices
        best_index_position = valid_indices.get_loc(best_epoch)
        
        # Get the indices of the AVERAGE_EPOCHS epochs to be averaged
        idx_adjustment = (average_epochs - 1) // 2
        start_index = max(0, best_index_position - idx_adjustment)
        end_index = min(len(valid_indices), best_index_position + idx_adjustment + 1)
        selected_indices = valid_indices[start_index:end_index]
        
        # Average the metrics over the selected epochs
        averaged_metrics = history.loc[selected_indices].mean() # TOOD: Fix error
        
        print(f"Run {run.url} - Best epoch: {best_epoch} (averaged over {average_epochs}) - {metric_name}: {averaged_metrics[metric_name]}")
        return averaged_metrics, best_epoch
    else:
        print(f"Run {run.url} - Best epoch: {best_epoch} - {metric_name}: {best_metrics[metric_name]}")
        return best_metrics, best_epoch

def gather_metrics_for_experiment_group(experiment_group, average_epochs=False, has_weighted_accuracy=True):
    api = wandb.Api()

    # Fetch runs matching the experiment group
    runs = api.runs("crop-classification/messis", filters={"config.experiment_group": experiment_group})
    print(f"Found {len(runs)} runs for experiment group '{experiment_group}'")

    # Organize the runs by setup name
    setups = {}
    for run in runs:
        if run.state != 'finished':
            print(f"Run {run.url} - Skipping (not finished, state: {run.state})")
            continue
        if 'name' not in run.config:
            if 'experiment_name' in run.config:
                setup_name = run.config['experiment_name'].split('-')[-1]
                print(f"Run {run.url} - Missing 'name' in config, getting it from 'experiment_name' instead: {setup_name}")
            else:
                print(f"Run {run.url} - Missing 'name' in config, skipping")
                continue
        else:
            setup_name = run.config['name']
        if setup_name not in setups:
            setups[setup_name] = {
                'runs': [],
                'metrics': {metric: [] for metric in [
                    'val_f1_tier1_majority',
                    'val_f1_tier2_majority',
                    'val_f1_tier3_majority',
                    'val_weighted_accuracy_tier1_majority' if has_weighted_accuracy else 'val_accuracy_tier1_majority',
                    'val_weighted_accuracy_tier2_majority' if has_weighted_accuracy else 'val_accuracy_tier2_majority',
                    'val_weighted_accuracy_tier3_majority' if has_weighted_accuracy else 'val_accuracy_tier3_majority',
                    'val_precision_tier1_majority',
                    'val_precision_tier2_majority',
                    'val_precision_tier3_majority',
                    'val_recall_tier1_majority',
                    'val_recall_tier2_majority',
                    'val_recall_tier3_majority',
                    'val_cohen_kappa_tier1_majority',
                    'val_cohen_kappa_tier2_majority',
                    'val_cohen_kappa_tier3_majority',
                ]},
                'best_steps': []
            }
        setups[setup_name]['runs'].append(run)

    # Print setup names with number of runs
    for setup_name, setup_data in setups.items():
        print(f"Setup '{setup_name}' - {len(setup_data['runs'])} runs")

    # Gather metrics for each setup
    for setup_name, setup_data in setups.items():
        for run in setup_data['runs']:
            best_metrics, best_epoch = get_best_epoch_metrics(run, 'val_f1_tier3_majority', average_epochs)
            setup_data['best_steps'].append(best_epoch)
            for metric in setup_data['metrics']:
                setup_data['metrics'][metric].append(best_metrics[metric])
    
    # Prepare data for DataFrame
    data = []
    for setup_name, setup_data in setups.items():
        metrics_summary = {
            'setup': setup_name,
            'num_runs': len(setup_data['runs']),
            'average_epochs': average_epochs,
            'run_handles': [run.id for run in setup_data['runs']],
            'best_steps': setup_data['best_steps'],
        }
        for metric, values in setup_data['metrics'].items():
            metrics_summary[f'{metric}_mean'] = np.mean(values)
            metrics_summary[f'{metric}_std'] = np.std(values)
        data.append(metrics_summary)
    
    metrics_df = pd.DataFrame(data)
    
    return metrics_df

# Run the function for a given experiment group
metrics_df = gather_metrics_for_experiment_group(EXPERIMENT_GROUP, average_epochs=AVERAGE_EPOCHS, has_weighted_accuracy=HAS_WEIGHTED_ACCURACY)

# Save the DataFrame to a CSV file
metrics_df.to_csv(f"experiment_results/eval_{EXPERIMENT_GROUP}.csv", index=False)

metrics_df

Found 7 runs for experiment group 'exp-5-data-augmentation'
Run https://wandb.ai/crop-classification/messis/runs/ef106jwj - Skipping (not finished, state: running)
Run https://wandb.ai/crop-classification/messis/runs/7pm6kx4y - Skipping (not finished, state: running)
Setup 'no-aug' - 3 runs
Setup 'f-0.5-j-0.01' - 2 runs
Run https://wandb.ai/crop-classification/messis/runs/gohfdvbc - Best epoch: 3200 - val_f1_tier3_majority: 0.20690637826919556
Run https://wandb.ai/crop-classification/messis/runs/2ius5tmk - Best epoch: 4858 - val_f1_tier3_majority: 0.2122039794921875
Run https://wandb.ai/crop-classification/messis/runs/d6680sg9 - Best epoch: 4154 - val_f1_tier3_majority: 0.2156532108783722
Run https://wandb.ai/crop-classification/messis/runs/wmnirn4w - Best epoch: 5059 - val_f1_tier3_majority: 0.2542308568954468
Run https://wandb.ai/crop-classification/messis/runs/iflsz305 - Best epoch: 5260 - val_f1_tier3_majority: 0.2545701563358307


,setup,num_runs,average_epochs,run_handles,best_steps,val_f1_tier1_majority_mean,val_f1_tier1_majority_std,val_f1_tier2_majority_mean,val_f1_tier2_majority_std,val_f1_tier3_majority_mean,...,val_recall_tier2_majority_mean,val_recall_tier2_majority_std,val_recall_tier3_majority_mean,val_recall_tier3_majority_std,val_cohen_kappa_tier1_majority_mean,val_cohen_kappa_tier1_majority_std,val_cohen_kappa_tier2_majority_mean,val_cohen_kappa_tier2_majority_std,val_cohen_kappa_tier3_majority_mean,val_cohen_kappa_tier3_majority_std
0,no-aug,3,None,"[gohfdvbc, 2ius5tmk, d6680sg9]","[3200, 4858, 4154]",0.466420,0.062252,0.308958,0.015768,0.211588,...,0.301030,0.019258,0.202019,0.007167,0.866168,0.002768,0.736748,0.006691,0.687858,0.008042
1,f-0.5-j-0.01,2,None,"[wmnirn4w, iflsz305]","[5059, 5260]",0.493552,0.063434,0.327471,0.010838,0.254401,...,0.316826,0.018490,0.236882,0.004034,0.858623,0.005036,0.748219,0.009753,0.709354,0.010471


In [14]:
# Mapping for short metric names
metric_name_mapping = {
    'val_f1_tier1_majority': 'F1 $T_1$',
    'val_f1_tier2_majority': 'F1 $T_2$',
    'val_f1_tier3_majority': 'F1 $T_3$',
    'val_precision_tier1_majority': 'Precision $T_1$',
    'val_precision_tier2_majority': 'Precision $T_2$',
    'val_precision_tier3_majority': 'Precision $T_3$',
    'val_recall_tier1_majority': 'Recall $T_1$',
    'val_recall_tier2_majority': 'Recall $T_2$',
    'val_recall_tier3_majority': 'Recall $T_3$',
    'val_cohen_kappa_tier1_majority': 'Kappa $T_1$',
    'val_cohen_kappa_tier2_majority': 'Kappa $T_2$',
    'val_cohen_kappa_tier3_majority': 'Kappa $T_3$',
}

if HAS_WEIGHTED_ACCURACY:
    metric_name_mapping['val_weighted_accuracy_tier1_majority'] = 'W. Acc. $T_1$'
    metric_name_mapping['val_weighted_accuracy_tier2_majority'] = 'W. Acc. $T_2$'
    metric_name_mapping['val_weighted_accuracy_tier3_majority'] = 'W. Acc. $T_3$'
else:
    metric_name_mapping['val_accuracy_tier1_majority'] = 'Acc. $T_1$'
    metric_name_mapping['val_accuracy_tier2_majority'] = 'Acc. $T_2$'
    metric_name_mapping['val_accuracy_tier3_majority'] = 'Acc. $T_3$'

# Function to create a LaTeX table with bold best scores and underlined second-best scores
def create_latex_table(df, baseline_setup, first_n_metrics=None):
    metrics = list(metric_name_mapping.keys())
    if first_n_metrics:
        metrics = metrics[:first_n_metrics]
    
    # Determine the best and second-best scores for each metric
    best_scores = {metric: df[f'{metric}_mean'].max() for metric in metrics}
    second_best_scores = {
        metric: sorted(df[f'{metric}_mean'].unique(), reverse=True)[1] if len(df[f'{metric}_mean'].unique()) > 1 else None
        for metric in metrics
    }

    # Reorder the dataframe to have the baseline setup first
    df = df.set_index('setup')
    if baseline_setup in df.index:
        df = df.loc[[baseline_setup] + [idx for idx in df.index if idx != baseline_setup]].reset_index()
    else:
        df = df.reset_index()

    # Initialize the Texttable object
    table = tt.Texttable()
    
    # Define the header
    headers = ['Metric'] + [f"{'$\\dagger$ ' if row['setup'] == baseline_setup else ''}{setup} ({row['num_runs']}-fold)" 
                            for idx, row in df.iterrows() for setup in [row['setup']]]
    table.header(headers)
    
    # Set alignment for columns
    cols_align = ["c"] * len(headers)
    cols_align[0] = "l"
    table.set_cols_align(cols_align)
    
    # Populate the table with data
    for metric in metrics:
        row_data = [metric_name_mapping[metric]]
        for idx, row in df.iterrows():
            mean = row[f'{metric}_mean'] * 100
            std = row[f'{metric}_std'] * 100
            value = f'{mean:.1f}\\% ± {std:.1f}\\%'
            if mean == best_scores[metric] * 100:
                value = f'\\textbf{{{value}}}'
            elif second_best_scores[metric] and mean == second_best_scores[metric] * 100:
                value = f'\\underline{{{value}}}'
            row_data.append(value)
        table.add_row(row_data)
    
    # Generate the LaTeX table
    average_epochs_used = df['average_epochs'].iloc[0]
    caption_averaged =  f"averaged over {average_epochs_used} epochs" if average_epochs_used else ""
    latex_table = latextable.draw_latex(
        table, 
        caption=f"Results for experiment {EXPERIMENT_GROUP}. All metrics calculated on field majority during validation phase {caption_averaged}, with mean and std reported. Best scores bold, second best underlined, $\\dagger$ is baseline.", 
        label=f"tab:metrics_comparison_{EXPERIMENT_GROUP}", 
        use_booktabs=True,
        position="h"
    )

    # Add \resizebox to the LaTeX table
    latex_table = latex_table.replace('\\begin{center}', '\\begin{center}\\resizebox{\\textwidth}{!}{')
    latex_table = latex_table.replace('\\end{center}', '}\\end{center}')

    return latex_table

# Generate the LaTeX table
df = pd.read_csv(f"experiment_results/eval_{EXPERIMENT_GROUP}.csv")
df = df.sort_values(by='setup')

# Optional: Drop rows to exclude some setups
# df = df.drop([0, 1])

latex_table = create_latex_table(df, baseline_setup=BASELINE_SETUP, first_n_metrics=None)
print(latex_table)

\begin{table}[h]
	\begin{center}\resizebox{\textwidth}{!}{
		\begin{tabular}{lcc}
			\toprule
			Metric & $\dagger$ no-aug (3-fold) & f-0.5-j-0.01 (2-fold) \\
			\midrule
			F1 $T_1$ & \underline{46.6\% ± 6.2\%} & \textbf{49.4\% ± 6.3\%} \\
			F1 $T_2$ & \underline{30.9\% ± 1.6\%} & \textbf{32.7\% ± 1.1\%} \\
			F1 $T_3$ & \underline{21.2\% ± 0.4\%} & \textbf{25.4\% ± 0.0\%} \\
			Precision $T_1$ & \underline{52.6\% ± 7.7\%} & \textbf{56.9\% ± 1.2\%} \\
			Precision $T_2$ & \underline{34.0\% ± 2.0\%} & \textbf{37.3\% ± 1.3\%} \\
			Precision $T_3$ & \underline{24.8\% ± 0.8\%} & \textbf{32.2\% ± 0.7\%} \\
			Recall $T_1$ & \underline{46.0\% ± 7.4\%} & \textbf{48.8\% ± 8.6\%} \\
			Recall $T_2$ & \underline{30.1\% ± 1.9\%} & \textbf{31.7\% ± 1.8\%} \\
			Recall $T_3$ & \underline{20.2\% ± 0.7\%} & \textbf{23.7\% ± 0.4\%} \\
			Kappa $T_1$ & \textbf{86.6\% ± 0.3\%} & \underline{85.9\% ± 0.5\%} \\
			Kappa $T_2$ & \underline{73.7\% ± 0.7\%} & \textbf{74.8\% ± 1.0\%} \\
			Kappa $T_3$ & \un